In [146]:
import pandas as pd
import numpy as np

In [330]:
def load_data():
    df=pd.read_csv("olympics.csv", skiprows=1)
    df.index = [s.split("\xa0")[0] for s in df[df.columns[0]]]
    df.index.rename("Country_Name")
    cols = pd.MultiIndex(labels=[[0,0,0,0,0,1,1,1,1,1,2,2,2,2,2,3],[0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,5]], 
                     levels=[['Summer', 'Winter','Combined','Country_Code'],['Games','Gold','Silver','Bronze','Total_Medals','']],
                     names=["Game_season", "Performance"])
    
    def getCountryCode(s):
        import re
        code = re.search(r'\([A-Z]{3,}\)',s)
        return ("" if type(code) == None.__class__ else code.group(0).lstrip("(").rstrip(")"))
    
    df["Country_Code"] = [getCountryCode(s) for s in df[df.columns[0]]]
    df.drop(df.columns[[0]], axis=1, inplace=True)
    df.drop('Totals', inplace=True)
    df.columns = cols
    df.drop(("Total_Medals"), level=1, axis=1, inplace=True)
    return(df)

In [331]:
df = load_data()

In [372]:
#df.head(10)

In [370]:
#%timeit abs((df["Summer"]["Gold"] - df["Winter"]["Gold"]).astype(int)).argmax()


In [371]:
#%timeit np.abs((df["Summer"]["Gold"] - df["Winter"]["Gold"]).astype(int)).argmax()

In [369]:
#Which country had the biggest difference between their summer and winter gold medal counts?
#df.loc[np.abs((df["Summer"]["Gold"] - df["Winter"]["Gold"]).astype(int)).argmax()]

In [368]:
#df.loc['United States']

In [366]:
#df["Combined"]["Gold"]*3 + df["Combined"]["Silver"]*2 + df["Combined"]["Bronze"]

In [367]:
#from scipy.spatial.distance import cdist
df = df.drop("Country_Code", axis=1)
df.head()

Game_season Summer                    Winter                    Combined       \
Performance  Games Gold Silver Bronze  Games Gold Silver Bronze    Games Gold   
Afghanistan     13    0      0      2      0    0      0      0       13    0   
Algeria         12    5      2      8      3    0      0      0       15    5   
Argentina       23   18     24     28     18    0      0      0       41   18   
Armenia          5    1      2      9      6    0      0      0       11    1   
Australasia      2    3      4      5      0    0      0      0        2    3   

Game_season                
Performance Silver Bronze  
Afghanistan      0      2  
Algeria          2      8  
Argentina       24     28  
Armenia          2      9  
Australasia      4      5

In [377]:
#Scaling the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df)
scaled_features = scaler.transform(df)
df_feat = np.array(scaled_features)
df_feat

array([[-0.0549309 , -0.34163707, -0.40692753, ..., -0.36061089,
        -0.41924295, -0.44270597],
       [-0.19814361, -0.28977701, -0.38204317, ..., -0.31497185,
        -0.39788957, -0.37568031],
       [ 1.37719618, -0.15494083, -0.10831516, ..., -0.19631036,
        -0.16300236, -0.15226147],
       ..., 
       [-0.19814361, -0.34163707, -0.39448535, ..., -0.36061089,
        -0.40856626, -0.45387691],
       [-0.19814361, -0.31052103, -0.35715881, ..., -0.33322746,
        -0.37653619, -0.45387691],
       [-1.48705798, -0.25866097, -0.34471662, ..., -0.28758843,
        -0.3658595 , -0.42036408]])

In [381]:
from scipy.spatial import distance
distance.euclidean(df_feat[0], df_feat[1])

0.4777685441919582

In [384]:
np.random.uniform(low=-1, high=1, size=(12,))

array([-0.88318588,  0.97325481, -0.12219377, -0.95496971, -0.67161267,
        0.49927629,  0.13307681, -0.81994919,  0.07168957, -0.39405817,
        0.10796521, -0.12265456])

In [391]:
c_array = np.array([np.zeros(12), np.ones(12)])
c_array

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]])

In [459]:
#distance.pdist(df_feat[0:10]).argmax()
mx = distance.pdist(df_feat[0:10]).max()

array([-0.0549309 , -0.34163707, -0.40692753, -0.43116881, -0.90061024,
       -0.34735569, -0.35385783, -0.37232179, -0.53526785, -0.36061089,
       -0.41924295, -0.44270597])

In [466]:
#set_size = df_feat.shape[0]*c_array.shape[0]
set_size = df_feat.shape[0]
dists = pd.DataFrame(data={}, columns=["point", "centroid", "distance"], index=np.arange(0,set_size))
#i=np.arange(0,df_feat.shape[0]*c_array.shape[0])
dists["point"] = [df_feat[i] for i in np.arange(0,set_size)]
dists["centroid"] = [np.zeros(12) for i in np.arange(0,set_size)]
dists["distance"] = mx

In [469]:

i=0
for point in df_feat:
    for centroid in c_array:
        #print(point, centroid, distance.euclidean(point, centroid))
        new_dist = distance.euclidean(point, centroid)
        if dists["distance"][i] > new_dist:
            dists["distance"][i] = new_dist
            dists["point"][i] = point
            dists["centroid"][i] = centroid
        i = i+1

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


KeyError: 146

In [468]:
dists

,point,centroid,distance
0,"[-0.0549309016397, -0.341637073064, -0.4069275...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.566885
1,"[-0.0549309016397, -0.341637073064, -0.4069275...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",4.938462
2,"[-0.198143609486, -0.289777005829, -0.38204317...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.250570
3,"[-0.198143609486, -0.289777005829, -0.38204317...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",4.698930
4,"[1.37719617682, -0.154940831015, -0.1083151615...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.727152
5,"[1.37719617682, -0.154940831015, -0.1083151615...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",3.771863
6,"[-1.20063256441, -0.331265059617, -0.382043170...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.758679
7,"[-1.20063256441, -0.331265059617, -0.382043170...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",5.051324
8,"[-1.63027068795, -0.310521032723, -0.357158805...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.559824
9,"[-1.63027068795, -0.310521032723, -0.357158805...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",5.733832
